# Iterative Dichotomiser 3 (ID3) 

## Implementation

Importing necessary libraries

In [ ]:
import math
import numpy as np
import pandas as pd
import time

### Calculate Entropy
#### Input
list of probabilities. 
#### Return
The entropy =  $\sum_{i=0}^{N-1} -P_i$ * $log$<sub>2</sub>$P_i$

#### Try except statement
To handle the case of $P=0$ 

In [ ]:
def calculate_entropy(p):
    e = 0
    for pi in p:
        try:
            e = e + -pi * math.log(pi, 2)
        except ValueError:
            e = e + 0
    return e

### Calculate Global Entropy
Calculates the golbal entropy for a given dataset.
#### Input
Pandas dataframe
#### Return
Float represents the global entropy.

In [ ]:
def calculate_global_entropy(dataset):
    classes = get_class_column(dataset)
    unique_classes = classes.unique()
    p = []
    for c in unique_classes:
        p.append(calculate_probability(c, classes))
    return calculate_entropy(p)

### Feature-Value-Information-Map
Takes the class column and a feature column and creates a dictionary that maps the feature values {key_1} to another dictionary {value_1} that maps the classes {key_2} to number of records that belongs to this class{value_1}. This is a step to calculate the information gain. 
#### Input
feature_values: pandas dataframe of one column
class_values: pandas dataframe of one column
#### Return
feature_value_information_map

In [ ]:
def feature_value_information_map(feature_values, class_values):
    classes = class_values.unique()
    unique_feature_values = feature_values.unique()
    feature_info_map = dict()
    for f in unique_feature_values:
        feature_info_map[f] = dict()
    for f in unique_feature_values:
        for c in classes:
            _myMap = feature_info_map[f]
            _myMap[c] = 0
    for f, c in zip(feature_values, class_values):
        _myMap = feature_info_map[f]
        _myMap[c] = _myMap[c] + 1
    return feature_info_map

### Calculate Information Gain
#### Input 
dataset: pandas dataframe
feature_name: string of the feature name.
#### Return
Information gain

In [ ]:
def calculate_information_gain(dataset, feature_name):
    feature_values = dataset[feature_name]
    class_values = get_class_column(dataset)
    unique_feature_values = feature_values.unique()
    classes = class_values.unique()
    f_v_info_map = feature_value_information_map(feature_values, class_values)
    entropy = []
    summ = []
    for f in unique_feature_values:
        summation = 0
        p = []
        for c in classes:
            _myMap = f_v_info_map[f]
            summation = summation + _myMap[c]
        for c in classes:
            _myMap = f_v_info_map[f]
            val = _myMap[c]
            p.append(val / summation)
        summ.append(summation)
        entropy.append(calculate_entropy(p))
    weighted_sum = 0
    for i in range(len(entropy)):
        weighted_sum = weighted_sum + (entropy[i] * (summ[i] / len(feature_values)))
    global_entropy = calculate_global_entropy(dataset)
    information_gain = global_entropy - weighted_sum
    return information_gain

### Choose Root
Chooses the feature that has the highest information gain.
#### Input
dataset : pandas dataframe
#### Return
String : name of the most informative feature.
##### Note
In case of two equal information gains (and this value is the maximum), The function returns the first feature in order in the dataframe. 

In [ ]:
def choose_root(dataset):
    features = get_features(dataset)
    information_gain = []
    for f in features:
        information_gain.append(calculate_information_gain(dataset, f))
    ind = information_gain.index(np.max(information_gain))
    return np.asarray(features)[ind]

#### Value-Data-Map
Splits the dataset into a map according to the different classes (values) of a certain feature.
{key = feature value : value = dataframe of the dataset} 
##### Input 
dataset : pandas dataframe
feature : string of feature name
##### Return
value_data_map

In [ ]:
def value_data_map(dataset, feature):
    class_column_list = dataset[feature]
    classes = class_column_list.unique()
    class_column_list = list(class_column_list)
    class_data_map = dict()
    for c in classes:
        class_data_map[c] = []
    for c in classes:
        for i in range(len(class_column_list)):
            if class_column_list[i] == c:
                class_data_map[c].append(dataset.iloc[i])
    for c in classes:
        class_data_map[c] = pd.DataFrame(class_data_map[c])
        class_data_map[c] = class_data_map[c].drop(feature, axis=1)
    return class_data_map

### Can Decide
Decide if it is possible to predict the class of a record (sample) or not.
#### Input
record : pandas dataframe
val_data_map : the dictionary returned from the value_data_map function
root : String of the most informative feature.
#### Return
True : if all class values belong to the same class.
False : if it is possible to make more iterations toward less uncertainty.
Prob : String that is a flag to choose a class of the highest probability.

In [ ]:
def can_decide(record, val_data_map, root):
    c = np.asarray(record[root])[0]
    data = val_data_map[c]
    classes = get_class_column(data)
    features = get_features(data)
    if len(features) == 0:
        return 'prob'
    elif len(classes.unique()) > 1:
        return False
    else:
        return True

### Predict 
#### Input
dataset : pandas dataframe
record : pandas dataframe of one sample
#### Return
return the class if certainty is one or float of probability of class equals one.

In [ ]:
def predict(dataset, record):
    root = choose_root(dataset)
    v_d_map = value_data_map(dataset, root)
    c = np.asarray(record[root])[0]
    try:  # some error rise say that this class doesn't exist in the dictionary.
        data = v_d_map[c]
    except:
        prob = calculate_probability(1, get_class_column(dataset))
        return prob
    if can_decide(record, v_d_map, root) == 'prob':
        prob = calculate_probability(1, get_class_column(data))
        return prob
    if can_decide(record, v_d_map, root):
        return list(get_class_column(data))[0]
    else:
        return predict(data, record)

## Data Cleaning
#### Transform continuous attributes into discrete 
* Turn "age" attribute into 5 classes based on values.
* Turn "height" attribute into 4 classes based on values.
* Turn "weight" attribute into 4 classes based on values.
#### "ap_hi" attribute
* Remove negative values.
* Edit unaccepted values that ranges form 1 to 25 by a proper multiplication
* Edit unaccepted values that is ranges from 700 to 16000 by a proper devision.
* Edit two values of 309 and 401 as they are unique.
#### "ap_lo" attribute
* Remove negative values.
* Edit unaccepted values that is ranges from 500 to max by a proper devision.
* Replace unaccepted (very low) values with the mean to reduce number of classes.
* Reduce number of classes to iclude only multiplications of 10 values.

## Ensemble Learning
Divide the training dataset into 5 groups. And apply the ID3 algorithm to each testing record with respect to each group of data and decide the class based on the majority.

In [ ]:
train_set_1 = cardio_data[1000:3000]
train_set_2 = cardio_data[3000:5000]
train_set_3 = cardio_data[5000:7000]
train_set_4 = cardio_data[7000:9000]
train_set_5 = cardio_data[9000:11000]
test_set = cardio_data[0:999]
test_set_labels = np.asarray(test_set.pop('cardio'))
result_labels = []
start_time = time.time()
for i in range(len(test_set)):
    decision = 0
    p1 = predict(train_set_1, test_set.iloc[[i]]))
    if p1 < 0.5:
        decision = decision - 1
    else :
        decision = decision + 1
    p2 = predict(train_set_2, test_set.iloc[[i]]))
    if p2 < 0.5:
        decision = decision - 1
    else :
        decision = decision + 1
    p3 = predict(train_set_3, test_set.iloc[[i]]))
    if p3 < 0.5:
        decision = decision - 1
    else :
        decision = decision + 1
    p4 = predict(train_set_4, test_set.iloc[[i]]))
    if p4 < 0.5:
        decision = decision - 1
    else :
        decision = decision + 1
    p5 = predict(train_set_5, test_set.iloc[[i]]))
    if p5 < 0.5:
        decision = decision - 1
    else :
        decision = decision + 1
    if decision < 0:
        result_labels.append(0)
    else :
        result_labels.append(1)
accuracy = 0
for i in range(len(result_labels)):
    if result_labels[i] == test_set_labels[i]:
        accuracy = accuracy + 1
print(accuracy/len(result_labels))
result_df = pd.DataFrame(zip(result_labels, test_set_labels),
                         columns=['Result labels', 'Actual labels'])
compression_opts = dict(method='zip', archive_name='result.csv')
result_df.to_csv('results.zip', index=False, compression=compression_opts)
print("--- %s seconds ---" % (time.time() - start_time))